### Project 2
### Name: Tao Jiang (Tommy)
### Student ID: 20004769

# Data Acquisition

* ##  Loading libraries

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup


* ##  Players Stats

> ### The data Scraped from the basketball-reference website, and the data source is trustworthy and accurateness of the site. I select the data from 2019-2020 NBA Player Totals Stats to analyze.

>https://www.basketball-reference.com/leagues/NBA_2020_totals.html

In [2]:
year = 2020
url = "https://www.basketball-reference.com/leagues/NBA_2020_totals.html"
soup = BeautifulSoup( urlopen(url))

In [3]:
tr =  soup.findAll('tr', limit=2)
th = tr[0].findAll('th')
headers = [th.getText() for th in th]
headers = headers[1:]
records = soup.findAll('tr')
records = records[1:]
player_stats = []
for i in range(len(records)):
    temp = []
    for td in records[i].findAll('td'):
        temp.append (td.getText())
    player_stats.append(temp)

In [4]:
players_stats = pd.DataFrame(player_stats, columns = headers)
players_stats.shape

(649, 29)

In [5]:
players_stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,58,58,1564,262,443,.591,...,.590,196,347,543,141,50,65,86,111,633
1,Bam Adebayo,PF,22,MIA,65,65,2235,408,719,.567,...,.690,165,518,683,333,78,85,185,164,1053
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,.493,...,.827,103,289,392,129,36,87,74,128,1001
3,Nickeil Alexander-Walker,SG,21,NOP,41,0,501,77,227,.339,...,.607,8,72,80,74,11,7,40,46,211
4,Grayson Allen,SG,24,MEM,30,0,498,79,176,.449,...,.857,5,61,66,43,6,1,23,36,221


In [6]:
players_stats.to_csv("stats.csv.gz", compression='gzip')

* ## Players Salary

> ### ESPN is an American primary sports channel, and it records each season NBA Player Salaries authoritatively. I select the season of 2019-2020 salary data.


> http://www.espn.com/nba/salaries

In [7]:
url = "http://www.espn.com/nba/salaries"
soup = BeautifulSoup(urlopen(url))

In [8]:
totoal_pages_num = soup.select('.page-numbers')
pages = totoal_pages_num[0].text[-2:]
pages

'14'

In [9]:
pages = int(pages)
arr=[]
for page in range(1,pages+1):
    page_url =  "http://www.espn.com/nba/salaries/_/page/{}".format(page)
    soup = BeautifulSoup(urlopen(page_url))
    records = soup.findAll('tr', limit=1)[0]
    records = records.findAll('td')
    headers =[]
    for i in records:
        headers.append(i.getText() )

    rows = soup.findAll('tr')[1:]
    player_stats = []
    for i in range(len(rows)):
        temp = []
        for td in rows[i].findAll('td'):
            temp.append (td.getText())
        player_stats.append(temp)
        
    arr.extend(player_stats)

In [10]:
salary = pd.DataFrame(arr, columns = headers)
salary.shape

(564, 4)

In [11]:
salary.head()

,RK,NAME,TEAM,SALARY
0,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758"
1,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482"
2,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482"
3,4,"John Wall, PG",Washington Wizards,"$38,199,000"
4,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000"


In [12]:
salary.to_csv("salary.csv.gz", compression='gzip')

In [25]:
import json


In [55]:
import requests

url = "https://free-nba.p.rapidapi.com/players"

querystring = {"page":"0","per_page":"25"}

headers = {
    'x-rapidapi-host': "free-nba.p.rapidapi.com",
    'x-rapidapi-key': "971070abcfmshfa96af7456eb43cp16031bjsn38eded1e11bf"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
# names = pd.DataFrame(response.text, columns = headers)

j = json.loads(response.text)
j['data']
names = pd.DataFrame(j['data'])

names = names[['id','first_name','last_name']]
# salary = raw_salary[['full-name','SALARY']]


In [56]:
names['temp'] = names['first_name'].apply(lambda x: x+ ' ')
names['full_name'] = names['temp'] +  names['last_name']


In [58]:
names = names[['id','full_name']]
names.head()

,id,full_name
0,14,Ike Anigbogu
1,25,Ron Baker
2,47,Jabari Bird
3,67,MarShon Brooks
4,71,Lorenzo Brown


In [59]:
import requests

url = "https://free-nba.p.rapidapi.com/players/71"

headers = {
    'x-rapidapi-host': "free-nba.p.rapidapi.com",
    'x-rapidapi-key': "971070abcfmshfa96af7456eb43cp16031bjsn38eded1e11bf"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{"id":71,"first_name":"Lorenzo","height_feet":null,"height_inches":null,"last_name":"Brown","position":"G","team":{"id":28,"abbreviation":"TOR","city":"Toronto","conference":"East","division":"Atlantic","full_name":"Toronto Raptors","name":"Raptors"},"weight_pounds":null}


In [14]:
import requests


In [13]:
# raw_salary['full-name'], raw_salary['Position'] = raw_salary['NAME'].str.split(',', 1).str
# raw_salary.head()

In [34]:
url = "https://free-nba.p.rapidapi.com/stats"
querystring = {"page":"0","per_page":"25"}
headers = {
    'x-rapidapi-host': "free-nba.p.rapidapi.com",
    'x-rapidapi-key': "971070abcfmshfa96af7456eb43cp16031bjsn38eded1e11bf"
    }
response = requests.request("GET", url, headers=headers, params=querystring)
j = json.loads(response.text)
j['data']
names = pd.DataFrame(j['data'])
names

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,game,min,oreb,pf,player,pts,reb,stl,team,turnover
0,1069008,0.0,1.0,2.0,0.200,5.0,1.0,0.333,9.0,3.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",20:08,0.0,0.0,"{'id': 415, 'first_name': 'Iman', 'height_feet...",7.0,2.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",2.0
1,1069009,4.0,0.0,5.0,0.000,2.0,0.0,0.200,5.0,1.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",19:22,1.0,1.0,"{'id': 49, 'first_name': 'Nemanja', 'height_fe...",2.0,6.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",0.0
2,1069010,4.0,1.0,5.0,0.000,0.0,0.0,0.667,6.0,4.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",27:24,6.0,2.0,"{'id': 91, 'first_name': 'Willie', 'height_fee...",8.0,11.0,3.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",2.0
3,1069011,1.0,0.0,1.0,0.545,11.0,6.0,0.500,18.0,9.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",32:06,0.0,2.0,"{'id': 210, 'first_name': 'Buddy', 'height_fee...",24.0,1.0,2.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",0.0
4,1069012,8.0,1.0,5.0,0.000,2.0,0.0,0.400,10.0,4.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",30:30,0.0,4.0,"{'id': 161, 'first_name': 'De'Aaron', 'height_...",10.0,5.0,1.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",4.0
5,1069013,2.0,0.0,1.0,0.500,2.0,1.0,0.615,13.0,8.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",27:20,0.0,2.0,"{'id': 53, 'first_name': 'Bogdan', 'height_fee...",17.0,1.0,1.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",2.0
6,1069014,2.0,0.0,1.0,0.333,3.0,1.0,0.400,5.0,2.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",16:22,0.0,1.0,"{'id': 157, 'first_name': 'Yogi', 'height_feet...",5.0,1.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",0.0
7,1069015,1.0,0.0,4.0,0.500,2.0,1.0,0.545,11.0,6.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",22:55,4.0,2.0,"{'id': 24, 'first_name': 'Marvin', 'height_fee...",15.0,8.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",3.0
8,1069016,2.0,0.0,3.0,0.250,4.0,1.0,0.400,5.0,2.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",24:07,0.0,2.0,"{'id': 235, 'first_name': 'Justin', 'height_fe...",6.0,3.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",0.0
9,1069017,4.0,0.0,4.0,0.000,0.0,0.0,0.000,3.0,0.0,...,"{'id': 45237, 'date': '2019-01-17T00:00:00.000...",16:14,0.0,6.0,"{'id': 174, 'first_name': 'Harry', 'height_fee...",1.0,4.0,0.0,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",4.0
